In [1]:
import os
from dataclasses import dataclass
from pathlib import Path

In [2]:
os.chdir("../")

In [3]:
%pwd

'd:\\text\\text-summarizer-project'

In [4]:
@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path
    data_path: Path
    model_path: Path
    tokenizer_path: Path
    metric_file_name: Path

In [5]:
from src.text_summarizer.constants import *
from text_summarizer.utils.common import read_yaml,create_dirs

In [11]:
class ConfigurationManager:
    def __init__(
        self,
        config_path=CONFIG_FILE_PATH,
        param_path=PARAM_FILE_PATH
    ):
        self.config=read_yaml(config_path)
        self.params=read_yaml(param_path)
        create_dirs([self.config.artifacts_root])
    def get_model_evaluation_config(self)->ModelEvaluationConfig:
        config=self.config.model_evaluation
        create_dirs([config.root_dir])
        model_evaluation_config=ModelEvaluationConfig(
                root_dir=config.root_dir,
                data_path=config.data_path,
                model_path=config.model_path,
                tokenizer_path=config.tokenizer_path,
                metric_file_name=config.metric_file_name
        )

In [8]:
from transformers import AutoTokenizer,AutoModelForSeq2SeqLM
from datasets import load_dataset,load_from_disk,load_metric
import torch
import pandas as pd
import tqdm

In [13]:
class ModelEvaluation:
    def __init__(self, config:ModelEvaluationConfig):
        self.config=config
    def generate_batch_sized_chunks(self,list_of_elements,batch_size):
        for i in range(0,len(list_of_elements),batch_size):
            yield list_of_elements[i:i+batch_size]
    def calculate_metric_on_test_ds(self,dataset,metric,model,tokenizer,
        batch_size=16,device="cuda" if torch.cuda.is_available() else "cpu",column_text="article",column_summary="highlights"
        ):
        article_batches=list(self.generate_batch_sized_chunks(dataset[column_text],batch_size))
        target_batches=list(self.generate_batch_sized_chunks(dataset[column_summary],batch_size))
        
        for article_batch,target_batch in tqdm(zip(article_batches,target_batches), total=len(article_batches)):
            inputs=tokenizer(article_batch,max_length=1024,truncation=True,padding="max_length",return_tensors="pt")
            summaries=model.generate(input_ids=inputs["input_ids"].to(device),attention_mask=input["attention_mask"].to(device),length_penalty=0.8,num_beams=0.8,max_length=128)

            decoded_summaries = [tokenizer.decode(s,skip_special_tokens=True,clean_up_tokenization_spaces=True) for s in summaries]
            decoded_summaries= [d.replace(""," ")  for d in decoded_summaries]
            metric.add_batch(prediction=decoded_summaries,references=target_batch)
        score=metric.compute()
        return score

    def evaluate(self):
        device = "cuda" if torch.cuda.is_available() else "cpu"
        tokenizer=AutoTokenizer.from_pretrained(self.config.tokenizer_path)
        model_pegasus = AutoModelForSeq2SeqLM.from_pretrained(self.config.model_path).to(device)
        samsum_dataset_t=load_from_disk(self.config.data_path)
        rouge_names = ["rouge1","rouge2","rougeL","rougeLsum"]
        rouge_metric=load_metric('rouge')
        score=self.calculate_metric_on_test_ds(samsum_dataset_t['test'][0:10],rouge_metric,model_pegasus,tokenizer,batch_size=2,column_text="dialogue",column_summary="summary")
        rouge_dict=dict((rn,score[rn].mid.fmeasure) for rn in rouge_names)
        df=pd.DataFrame(rouge_dict,index="pegasus")
        df.to_csv(self.config.metric_file_name,index=False)


In [15]:
try:
    config=ConfigurationManager()
    model_evaluation_config=config.get_model_evaluation_config()
    model_evaluation=ModelEvaluation(config=model_evaluation_config)
    model_evaluation.evaluate()
except Exception as e:
    raise e

[2023-08-24 12:11:21,352: INFO: common Successfully loaded config\config.yaml ,yaml file]
[2023-08-24 12:11:21,356: INFO: common Successfully loaded params.yaml ,yaml file]


AttributeError: 'NoneType' object has no attribute 'tokenizer_path'